In [2]:
from datetime import datetime
from pathlib import Path

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.decomposition import PCA
from torch.utils.data import DataLoader
from tqdm import tqdm, trange
import pandas as pd

from mimic import Mimic3, coallate
from model import PatientSimiEval
from siamese_cnn import SiameseCNN
from utils import find_nearest
from word2vec import SkipGramDataset, Word2Vec

# DATASET_PATH = '/content/drive/MyDrive/mimic3/'
# MODEL_PATH = '/content/drive/MyDrive/mimic3/saved_models/'
MODEL_PATH = './Models/'
DATASET_PATH = './Dataset/'
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.__version__

'1.11.0'

In [4]:
# load a trained PSE model

pse = {
    'name': 'pse_20220505-154958',
    'version': 'pse_epoch499'
}

pse_path = Path(MODEL_PATH) / pse['name'] / pse['version']
pse_model = torch.load(pse_path)

In [ ]:
# create the main dataset of medical concept sequences (preprocessing) 

dataset = Mimic3(DATASET_PATH,
                    apply_vocab=True,
                    to_tensor=False)

data_loader = DataLoader(dataset, 
                        batch_size=128,
                        collate_fn=coallate,
                        shuffle=False)

### Baselines

In [6]:
""" BASELINES """

# PSE
reprs_pse = np.vstack([
    pse_model(codes).detach().cpu().numpy() 
    for codes, _ in tqdm(data_loader)
    ])

nearest_pse = find_nearest(reprs_pse, metric='cosine')

    
# PCA: pca of one-hode encoded codes
codes_onehot = np.zeros((len(dataset), dataset.num_codes()), dtype=bool)
for i, (codes, _) in enumerate(dataset):
    codes_onehot[i, codes] = 1    
    
out_dim = pse_model.scnn.linear.out_features # 30 \
pca = PCA(n_components=out_dim) #  cohorts
reprs_pca = pca.fit_transform(codes_onehot) # (N, out_dim)
nearest_pca = find_nearest(reprs_pca, metric='euclidean')


# Hamming distance:
reprs_hamming = codes_onehot
nearest_hamming = find_nearest(reprs_hamming, metric='hamming')

100%|██████████| 185/185 [01:46<00:00,  1.74it/s]


### HRR & IRDM
The Hospital Readmission Rate (HRR)   
Incidence Rate Difference for Mortality (IRDM)


In [12]:
# The Hospital Readmission Rate (HRR)
nadmin = dataset.info['NADMISSIONS'].to_numpy()
def hospital_readmin_rate(nearest):
    return (((nadmin[nearest] == 1) & (nadmin == 1)) | ((nadmin[nearest] > 1) & (nadmin > 1))).mean()


# Incidence Rate Difference for Mortality (IRDM)
dischtime = dataset.info['DISCHTIME']
deathtime = dataset.info['DEATHTIME']
deathtime = pd.to_datetime(deathtime.apply(lambda x: x[0] if isinstance(x, pd.arrays.DatetimeArray) else x))


def incidence_rate(dischtime, deathtime):
    death_ct = (~deathtime.isna()).sum()
    diff = (deathtime.fillna(datetime(2022, 1, 1)) - dischtime).dt.days
    return death_ct / diff.sum()


def inc_rate_mortality(nearest):
    ir_p = incidence_rate(dischtime, deathtime)
    ir_sp = incidence_rate(dischtime.iloc[nearest], deathtime.iloc[nearest])
    return np.abs(ir_p - ir_sp)

In [13]:
methods = {
    "PSE": nearest_pse,
    "PCA": nearest_pca,
    "Hamming": nearest_hamming
}


irdm = {}
hrr = {}
for name, nearest in methods.items():
    irdm[name] = inc_rate_mortality(nearest)
    hrr[name] = hospital_readmin_rate(nearest)

In [26]:
hrr

{'PSE': 0.745236175359674,
 'PCA': 0.9491575775580359,
 'Hamming': 0.9287442176293341}

In [25]:
irdm

{'PSE': 4.418157087202884e-06,
 'PCA': 6.288749485720541e-08,
 'Hamming': 1.4706780777652385e-07}

### Disease Cohort Classification


In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

labels = dataset.labels

mlb = MultiLabelBinarizer()
y = mlb.fit_transform(labels)

methods = {
    "PSE": reprs_pse,
    "PCA": reprs_pca,
}

method_acc = {}
method_auc = {}

for name, x in methods.items():

    kf = KFold(n_splits=10)
    score = []
    auc = []
    for train_index, test_index in tqdm(kf.split(x)):
        x_train, x_test = x[train_index], x[test_index]
        y_train, y_test = y[train_index], y[test_index]

        clf = MLPClassifier().fit(x_train, y_train)
        # y_hat  = clf.predict(x_test)
        y_hat_prob = clf.predict_proba(x_test)
        auc.append(roc_auc_score(y_test, y_hat_prob))
        score.append(clf.score(x_test, y_test)) # provides mean accuracy

    method_acc[name] = score
    method_auc[name] = auc


In [30]:
print('Accuracy:')
for name, acc in method_acc.items():
    print(f'{name}: {np.mean(acc):.2f} +- {np.std(acc):.2f}')

Accuracy:
PSE: 0.07 +- 0.03
PCA: 0.53 +- 0.01


In [31]:
print('AUC')
for name, acc in method_auc.items():
    print(f'{name}: {np.mean(acc):.2f} +- {np.std(acc):.2f}')

AUC
PSE: 0.59 +- 0.03
PCA: 0.94 +- 0.00
